<a href="https://colab.research.google.com/github/NurFortuna/Deep_Learning_with_Tensorflow_notes/blob/main/Basics_of_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RNN
### Recurrent Neural Networks(RNN)'de kısaca bir sonraki katmanın çıktısı(output)  bir sonraki girdiye(input)' eklenir ve aynı katmana geri beslenir. 

### RNN'lerde girdiler birbiri ile ilişkilidir.

### RNN'ler önceki girdileri hatırlar ve her girdi için ilişki kurar.

### Belirli bir noktaya kadar yapılan analizi hatırlayabilir. RNN'nin hafızası olarak düşünebiliriz.

### Bir veri kümesindeki noktalar, veri kümesindeki diğer noktalara bağımlı olduğunda, verilerin sıralı veriler(Sequential data) olduğu söylenir.

### RNN sequential data'yı işleyebilen bir mekanizmaya sahiptir. 

![resim](https://miro.medium.com/max/1400/1*HgAY1lLMYSANqtgTgwWeXQ.png)

##LSTM

### LSTM  üç lojistik yapıdan oluşur. 

### LSTM yapısı içerisindeki kapılar(gate) neyin hatırlanacağını, neyin unutulacağını belirler. Yani gelen girdi önemsizse unutulur, Önemliyse bir sonraki aşamaya aktarılır.

### Forget gate(unutma kapısı): Hangi bilginin tutulacağını veya unutalacağına karar verir. 

### Input gate(girdi kapısı) Cell state'i güncellemek için kullanılır.

### Output gate(Çıktı kapısı) Bir sonraki katmana gönderilecek değere karar verir.



![resim](https://miro.medium.com/max/984/1*Mb_L_slY9rjMr8-IADHvwg.png)



In [1]:
!pip install tensorflow==2.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.6/454.6 KB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 KB 12.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: h5py
    Found existing installation: h

In [2]:
import numpy as np
import tensorflow as tf

Bir LSTM hücresine sahip bir network oluşturuyoruz. 

In [16]:
LSTM_CELL_SIZE = 4  

state = (tf.zeros([1,LSTM_CELL_SIZE]),)*2
state

(<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 0., 0., 0.]], dtype=float32)>)

In [17]:
lstm = tf.keras.layers.LSTM(LSTM_CELL_SIZE, return_sequences=True, return_state=True)

lstm.states=state

print(lstm.states)

(<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 0., 0., 0.]], dtype=float32)>)


In [18]:
#Batch size x time steps x features.
sample_input = tf.constant([[3,3,2,2,2,2]],dtype=tf.float32)

batch_size = 1
sentence_max_length = 1
n_features = 6

new_shape = (batch_size, sentence_max_length, n_features)

inputs = tf.constant(np.reshape(sample_input, new_shape), dtype = tf.float32)

In [19]:
output, final_memory_state, final_carry_state = lstm(inputs)

In [20]:
print('Output : ', tf.shape(output))

print('Memory : ',tf.shape(final_memory_state))

print('Carry state : ',tf.shape(final_carry_state))

Output :  tf.Tensor([1 1 4], shape=(3,), dtype=int32)
Memory :  tf.Tensor([1 4], shape=(2,), dtype=int32)
Carry state :  tf.Tensor([1 4], shape=(2,), dtype=int32)


Output shape'imiz (1,1,4) olarak çıktı. 1 batches, 1 dizimiz(sequence) elemente, 4 ise çıktının boyunu

## Stacked LSTM

In [11]:
cells = []

İlk layer(katman) LTSM'in oluşturulması

In [12]:
LSTM_CELL_SIZE_1 = 4 #4 hidden nodes
cell1 = tf.keras.layers.LSTMCell(LSTM_CELL_SIZE_1)
cells.append(cell1)

ikinci layer(katman) LTSM'in oluşturulması

In [13]:
LSTM_CELL_SIZE_2 = 5 #5 hidden nodes
cell2 = tf.keras.layers.LSTMCell(LSTM_CELL_SIZE_2)
cells.append(cell2)

Çok katmanlı LSTM oluşturmak içi `tf.keras.layers.StackedRNNCells` kullanırız.

In [14]:
stacked_lstm =  tf.keras.layers.StackedRNNCells(cells)

In [21]:
#Batch size x time steps x features.
sample_input = [[[1,2,3,4,3,2], [1,2,1,1,1,2],[1,2,2,2,2,2]],[[1,2,3,4,3,2],[3,2,2,1,1,2],[0,0,0,0,3,2]]]
sample_input

batch_size = 2
time_steps = 3
features = 6
new_shape = (batch_size, time_steps, features)

x = tf.constant(np.reshape(sample_input, new_shape), dtype = tf.float32)

In [22]:
output, final_memory_state, final_carry_state  = lstm_layer(x)

In [23]:
print('Output : ', tf.shape(output))

print('Memory : ',tf.shape(final_memory_state))

print('Carry state : ',tf.shape(final_carry_state))

Output :  tf.Tensor([2 3 5], shape=(3,), dtype=int32)
Memory :  tf.Tensor([2 2 4], shape=(3,), dtype=int32)
Carry state :  tf.Tensor([2 2 5], shape=(3,), dtype=int32)
